![Logo de AA1](logo_AA1_texto_small.png) 
# Sesión 06 - Preprocesado de datos categóricos

En esta sesión de prácticas vamos a ver cómo podemos abordar:
- la codificación de atributos categóricos y
- el tratamiento de valores deconocidos en atributos categóricos.




## 6.1 codificación de atributos categóricos

Muchas veces nos encontraremos con que los datos que tenemos para trabajar mezclan atributos numéricos y atributos categóricos y, además, esos atributos categóricos contendrán usualmente textos. Por ejemplo, podríamos tener un atributo cuyos posibles valores sean nombres de frutas (naranja, plátano, sandía) y otro atributo que represente su tamaño (pequeño, mediano, grande).

Aunque a primera vista puedan parecernos atributos de las mismas características, lo cierto es que hay una diferencias fundamental entre estos dos atributos: en el tamaño hay una relación de orden (pequeño es más parecido a mediano que a grande). Esto no sucede con el otro atributo.

Además, la mayoría de los algoritmos no pueden trabajar con textos y necesitaremos convertir esos textos a números.

### 6.1.1 Codificando atributos categóricos nominales

Vamos a empezar abordando el problema de los atributos categóricos nominales y para ello vamos a cargar la pestaña 'Datos' de la hoja de cálculo 'ejemplo.xlsx':

In [1]:
# se importan las librerías
import pandas as pd
from sklearn import preprocessing, impute

df = pd.read_excel('ejemplo.xlsx', sheet_name='Datos')
display(df)

,Fruta,Tamaño,Clase
0,naranja,grande,comprar
1,naranja,mediano,comprar
2,sandía,pequeño,no comprar
3,plátano,mediano,comprar
4,plátano,pequeño,no comprar
5,sandía,grande,comprar


Vemos dos atributos y la clase. El atributo 'Fruta' es un atributo categórico nominal ya que no hay una relación de orden entre los posibles valores que puede tomar. 

Si asignásemos un número a cada uno de los posible valores, por ejemplo 'naranja'->1, 'sandía->2' y 'plátano'->3, estaríamos realizando una mala codificación, ya que estaríamos indicando que 'plátano' está más próximo a 'sandía' que a 'naranja'.

Por tanto, debemos utilizar la codificación **one-hot**, que consiste en crear un nuevo atributo por cada uno de los posibles valores del atributo 'Fruta', de tal forma que un ejemplo tendrá ceros en las columnas que no se corresponden con el tipo de fruta que es y un uno en la columna que se corresponda con su tipo.

Así, si creamos las columnas ['naranja', 'plátano', 'sandía'], cada vez que nos encontremos con un plátano deberemos codificarlo como [0, 1, 0].

Hay muchas formas de realizar esto y cada librería tiene la suya propia. `Scikit-learn` nos da varias posibilidades y la más sencilla es utilizar la clase `LabelBinarizer()`:

In [2]:
# se crea el objeto LabelBinarizer
one_hot = preprocessing.LabelBinarizer()

# se leen los posibles valores de la columna fruta y se retornan tantas columnas como posibles valores con los ejemplos codificados
df[one_hot.classes_] = one_hot.fit_transform(df['Fruta'])

display(df)

# podría utilizarse también OneHotEncoder, que permite tratar vaios atributos a la vez, pero su uso es un poco más complicado
# enc = preprocessing.OneHotEncoder(handle_unknown='ignore', sparse=False)
# df[list(enc.categories_[0])] = enc.fit_transform(df['Fruta'].values.reshape(-1,1)).astype(int)

,Fruta,Tamaño,Clase,naranja,plátano,sandía
0,naranja,grande,comprar,1,0,0
1,naranja,mediano,comprar,1,0,0
2,sandía,pequeño,no comprar,0,0,1
3,plátano,mediano,comprar,0,1,0
4,plátano,pequeño,no comprar,0,1,0
5,sandía,grande,comprar,0,0,1


Fijaros en que el atributo `classes_` contendrá una lista con todos los tipos de fruta observados en la columna y que lo utilizaremos para dar nombre a las nuevas columnas creadas en el `DataFrame`. **El orden de las etiquetas que aparecen en `classes_` se corresponde con el orden de las columnas del one-hot**.

**IMPORTANTE:** `LabelBinarizer()` está pensado para trabajar con la clase, no con los atributos. Lo utilizamos en este punto por su sencillez pero lo correcto será utilizar `OneHotEncoder()` cuando se trate de atributos. En una práctica a final de curso, cuando tengamos un poco más de destreza, veremos cómo utilizar `OneHotEncoder()`.

Todas las clases de `scikit-learn` que utilizamos para transformar los datos tienen un método denominado `inverse_transform()` que nos permite realizar la transformación inversa. Así, utilizando este método podríamos volver a obtener los valores originales:

In [3]:
# le pasamos las columnas correspondientes en formato numpy (.values)
one_hot.inverse_transform(df[one_hot.classes_].values)

array(['naranja', 'naranja', 'sandía', 'plátano', 'plátano', 'sandía'],
      dtype='<U7')

La codificación one-hot es muy popular y casi todas las librerías tienen una función capaz de generar esa codificación. En la librería `pandas` podríamos haberlo hecho utilizando la función `get_dummies()`:

In [4]:
df = pd.read_excel('ejemplo.xlsx', sheet_name='Datos')

#  se retornan tantas columnas como posibles valores con los ejemplos codificados
fruta_one_hot = pd.get_dummies(df['Fruta'])

# añadimos las nuevas columnas al Dataframe
df[fruta_one_hot.columns] = fruta_one_hot

display(df)

,Fruta,Tamaño,Clase,naranja,plátano,sandía
0,naranja,grande,comprar,1,0,0
1,naranja,mediano,comprar,1,0,0
2,sandía,pequeño,no comprar,0,0,1
3,plátano,mediano,comprar,0,1,0
4,plátano,pequeño,no comprar,0,1,0
5,sandía,grande,comprar,0,0,1


En este caso, `get_dummies()` ya retorna un `Dataframe` y por tanto debemos acceder al campo `columns` para conocer el nombre de las columnas. Veremos en futuras sesiones que es mejor utilizar las clases de `scikit-learn` por motivos de compatibilidad con otras funcionalidades.

Lo hagamos de una manera o de la otra, podemos ver que ya no necesitamos la columna 'Fruta' puesto que tenemos otros atributos que ya recogen esa información, por tanto, deberíamos eliminarla:

In [5]:
# eliminamos la columna 'Fruta'
df = df.drop('Fruta', axis=1)

display(df)

,Tamaño,Clase,naranja,plátano,sandía
0,grande,comprar,1,0,0
1,mediano,comprar,1,0,0
2,pequeño,no comprar,0,0,1
3,mediano,comprar,0,1,0
4,pequeño,no comprar,0,1,0
5,grande,comprar,0,0,1


### 6.1.2 Codificando atributos categóricos ordinales

El atributo 'Tamaño' se trata de un atributo en el que los posibles valores tienen un sentido ordinal: grande es más parecido a mediano que a pequeño.

In [6]:
df = pd.read_excel('ejemplo.xlsx', sheet_name='Datos')

# se crea una lista con las categorías en orden
categ_tam = ['pequeño', 'mediano', 'grande']

# se vrea el objeto OrdinalEncoder con las 'categ_tam' entre [] ya que si hubiese otros 
# atributos ordinales podríamos tratarlos a la vez y tendríamos que pasarle las categorías de ese otro atributo también
ord_enc = preprocessing.OrdinalEncoder(categories=[categ_tam])

# llamamos a fit_transform convirtiendo los datos a un vector columna numpy: reshape(-1,1) - las filas que necesite y una columma
df['Tamaño'] = ord_enc.fit_transform(df['Tamaño'].values.reshape(-1,1)).astype('int')

display(df)

,Fruta,Tamaño,Clase
0,naranja,2,comprar
1,naranja,1,comprar
2,sandía,0,no comprar
3,plátano,1,comprar
4,plátano,0,no comprar
5,sandía,2,comprar


Hemos utilizado la clase `OrdinalEncoder()` de `Scikit-learn`. Como puede tratar varias columnas simultaneamente el parámetro `categories`, donde se indica el orden de las etiquetas, es una lista de listas.

Otra forma de hacer lo mismo de forma más sencilla y eficaz es mediante la función `replace()` de `pandas` que también nos permite indicar el valor que queremos darle a cada etiqueta:

In [7]:
df = pd.read_excel('ejemplo.xlsx', sheet_name='Datos')

# se crea un diccionario con el mapeo que consideremos adecuado
mapeo = {"pequeño":1, "mediano":2, "grande":3}

# se realiza la sustitución
df['Tamaño'] = df['Tamaño'].replace(mapeo)

display(df)

,Fruta,Tamaño,Clase
0,naranja,3,comprar
1,naranja,2,comprar
2,sandía,1,no comprar
3,plátano,2,comprar
4,plátano,1,no comprar
5,sandía,3,comprar


Esta forma de realizar la transformación es más versatil ya que nos da la posibilidad de asignar valores que no tengan por qué ser números consecutivos y que nos permitirán establecer diferencias más ajustadas entre las etiquetas, ya que podríamos establecer mapeos como:

In [8]:
mapeo = {"pequeño":1, "entre pequeño y mediano":1.5, "mediano":2, "grande":3, "inmenso no lo siguiente": 10}

display(mapeo)

{'pequeño': 1,
 'entre pequeño y mediano': 1.5,
 'mediano': 2,
 'grande': 3,
 'inmenso no lo siguiente': 10}



### 6.1.3 Codificando la clase

Si la clase es categórica y las etiquetas de la clase tienen un sentido ordinal, lo más sencillo es utilizar un mapeo con `replace()` como acabamos de ver.

Sin embargo, si no hay orden entre las diferentes clases podemos utilizar `LabelEncoder()`:

In [9]:
# se crea el objeto LabelEncoder
class_enc = preprocessing.LabelEncoder()

# se transforma la clase
df['Clase'] = class_enc.fit_transform(df['Clase'])

display(df)

,Fruta,Tamaño,Clase
0,naranja,3,0
1,naranja,2,0
2,sandía,1,1
3,plátano,2,0
4,plátano,1,1
5,sandía,3,0


Cuando la clase tiene **más de dos posibles etiquetas**, dependiendo del algoritmo de aprendizaje que vayamos a utilizar, puede ser que sirva con codificar las clases con números como acabamos de ver (0,1,2,3,...) utilizando `LabelEncoder()` o puede ser que necesitemos una codificación de tipo one-hot (por ejemplo en las redes neuronales).

### 6.1.4 Codificando atributos categóricos nominales (que tengan solo 2 posibles valores)

Si un atributo categórico nominal tiene únicamente 2 posibles valores (por ejemplo 'si' - 'no'), entonces no tiene sentido utilizar una columna para el 'si y otra para el 'no', ya que cuando en un caso pongamos un 1 en el otro habrá un 0. En estos casos podemos utilizar la misma columna reemplazando, por ejemplo, el 'no' con 0 y el 'si' con 1.

Para lograr esto, podremos utilizar `LabelEncoder()` o `replace()` con mapeo como hemos visto.

### 6.1.5 Todos los pasos 
Vamos a realizar todos los pasos para ver cómo quedaría el conjunto al final:

In [10]:
df = pd.read_excel('ejemplo.xlsx', sheet_name='Datos')

#  se retornan tantas columnas como posibles valores con los ejemplos codificados
fruta_one_hot = pd.get_dummies(df['Fruta'])

# añadimos las nuevas columnas al Dataframe
df[fruta_one_hot.columns] = fruta_one_hot

# eliminamos la columna 'Fruta'
df = df.drop('Fruta', axis=1)

# se crea un diccionario con el mapeo que consideremos adecuado
mapeo = {"pequeño":1, "mediano":2, "grande":3}

# se realiza la sustitución
df['Tamaño'] = df['Tamaño'].replace(mapeo)

# se crea el objeto LabelEncoder
class_enc = preprocessing.LabelEncoder()

# se transforma la clase
df['Clase'] = class_enc.fit_transform(df['Clase'])

display(df)

,Tamaño,Clase,naranja,plátano,sandía
0,3,0,1,0,0
1,2,0,1,0,0
2,1,1,0,0,1
3,2,0,0,1,0
4,1,1,0,1,0
5,3,0,0,0,1


### 6.1.6 Reordenando los atributos
El `Dataframe` resultante tiene la clase entre los atributos y esto es algo que a veces puede molestar o resultar un poco incómodo.

Para reordenar las columnas y dejar la clase como última columna, podemos hacer lo siguiente:

In [11]:
# indicamos la posición actual de la columna de la clase
colum_clase = 1

# creamos una lista con el nombre de las columnas en el orden que queramos
columnas = df.columns[:colum_clase].to_list() + df.columns[(colum_clase+1):].to_list() + df.columns[colum_clase:(colum_clase+1)].to_list()

print(columnas)

# reordenamos el dataframe
df.reindex(columns=columnas)

['Tamaño', 'naranja', 'plátano', 'sandía', 'Clase']


,Tamaño,naranja,plátano,sandía,Clase
0,3,1,0,0,0
1,2,1,0,0,0
2,1,0,0,1,1
3,2,0,1,0,0
4,1,0,1,0,1
5,3,0,0,1,0


## 6.2 Tratamiento de valores deconocidos en atributos categóricos

Al igual que sucede con los atributos contínuos (sesión anterior), a veces puede suceder que no se conozca alguno de los valores del conjunto. Cuando esto sucede, podemos optar por eliminar los ejemplos que tienen valores desconocidos utilizando `dropna()`(como ya vimos en la sesión anterior) o podemos tratar de asignar valores coherentes.

Lo más habitual es utilizar el `SimpleImputer()` con `strategy='most_frequent'` para asignar valores. De esta forma lo que estaríamos es asignado a los valores desconocidos la etiqueta más frecuente en el atributos. 

Debemos asegurarnos que la estrategia utilizada dé como resultado una de las etiquetas posibles, por tanto 'mean' y 'median' no se pueden utilizar:



In [12]:
df_simple_imp = pd.read_excel('ejemplo.xlsx', sheet_name='Missing', na_values='?')

print("\n#### Antes de asignar valores a los missing con SimpleImputer ####")
display(df_simple_imp)

# se crea un objeto de SimpleImputer con la media como estrategia
imputer_media = impute.SimpleImputer(strategy='most_frequent')

# se realiza el calculo de las medias y se asigna a los missing
df_simple_imp[df_simple_imp.columns] = imputer_media.fit_transform(df_simple_imp[df_simple_imp.columns])

print("\n#### Después de asignar valores a los missing con SimpleImputer ####")
display(df_simple_imp)


#### Antes de asignar valores a los missing con SimpleImputer ####


,Fruta,Tamaño,Clase
0,naranja,grande,comprar
1,naranja,mediano,comprar
2,sandía,pequeño,no comprar
3,plátano,mediano,comprar
4,NaN,pequeño,no comprar
5,sandía,NaN,comprar
6,naranja,NaN,no comprar
7,NaN,mediano,comprar
8,plátano,grande,no comprar



#### Después de asignar valores a los missing con SimpleImputer ####


,Fruta,Tamaño,Clase
0,naranja,grande,comprar
1,naranja,mediano,comprar
2,sandía,pequeño,no comprar
3,plátano,mediano,comprar
4,naranja,pequeño,no comprar
5,sandía,mediano,comprar
6,naranja,mediano,no comprar
7,naranja,mediano,comprar
8,plátano,grande,no comprar


Si quisiésemos utilizar el `KNNImputer()` deberíamos:
1. convertir todos los atributos a valores numéricos como hemos visto en esta sesión para que se puede calcular la distancia entre los ejemplos
2. utilizar el parámetro `n_neighbors=1` para que asigne al missing el valor del ejemplo más cercano (que será un valor válido). Si ponemos al parámetro un valor mayor que 1, entonces calculará la media de entre los ejemplos más cercanos y ya no sería un valor válido


In [13]:
df = pd.read_excel('ejemplo.xlsx', sheet_name='Missing', na_values='?')

#  se retornan tantas columnas como posibles valores con los ejemplos codificados
fruta_one_hot = pd.get_dummies(df['Fruta'])

# añadimos las nuevas columnas al Dataframe
df[fruta_one_hot.columns] = fruta_one_hot

# eliminamos la columna 'Fruta'
df = df.drop('Fruta', axis=1)

# se crea un diccionario con el mapeo que consideremos adecuado
mapeo = {"pequeño":1, "mediano":2, "grande":3}

# se realiza la sustitución
df['Tamaño'] = df['Tamaño'].replace(mapeo)

# se crea el objeto LabelEncoder
class_enc = preprocessing.LabelEncoder()

# se transforma la clase
df['Clase'] = class_enc.fit_transform(df['Clase'])

# indicamos la posición actual de la columna de la clase
colum_clase = 1

# creamos una lista con el nombre de las columnas en el orden que queramos
columnas = df.columns[:colum_clase].to_list() + df.columns[(colum_clase+1):].to_list() + df.columns[colum_clase:(colum_clase+1)].to_list()

display(columnas)

# reordenamos el dataframe
df = df.reindex(columns=columnas)

print("\n#### Antes de asignar valores a los missing con KNNImputer ####")
display(df)

# se crea un objeto de KNNImputer y n_neighbors DEBE SER 1
imputer_knn = impute.KNNImputer(n_neighbors=1)

# se realiza el calculo de las medias y se asigna a los missing
df[df.columns] = imputer_knn.fit_transform(df[df.columns])

print("\n#### Después de asignar valores a los missing con KNNImputer ####")
display(df)

['Tamaño', 'naranja', 'plátano', 'sandía', 'Clase']


#### Antes de asignar valores a los missing con KNNImputer ####


,Tamaño,naranja,plátano,sandía,Clase
0,3.0,1,0,0,0
1,2.0,1,0,0,0
2,1.0,0,0,1,1
3,2.0,0,1,0,0
4,1.0,0,0,0,1
5,NaN,0,0,1,0
6,NaN,1,0,0,1
7,2.0,0,0,0,0
8,3.0,0,1,0,1



#### Después de asignar valores a los missing con KNNImputer ####


,Tamaño,naranja,plátano,sandía,Clase
0,3.0,1.0,0.0,0.0,0.0
1,2.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,1.0
3,2.0,0.0,1.0,0.0,0.0
4,1.0,0.0,0.0,0.0,1.0
5,1.0,0.0,0.0,1.0,0.0
6,3.0,1.0,0.0,0.0,1.0
7,2.0,0.0,0.0,0.0,0.0
8,3.0,0.0,1.0,0.0,1.0


## Ejercicios

1. Carga el fichero **breast-cancer.data** (es un archivo de texto). Para realizar la carga debes tener en cuenta si tiene o no valores desconocidos y si no tiene cabecera debes asignar nombres a las columnas mediante el parámetro 'names'
2. En el '.names' podrás ver una descripción de los atributos y así entender su naturaleza
3. Trata los valores desconocidos
4. Convierte los atributos categóricos nominales como se ha explicado en el notebook
5. Convierte los atributos categóricos ordinales como se ha explicado en el notebook
6. Convierte la clase

Estos ejercicios no es necesario entregarlos.